### 项目背景

掘金在进行运营活动的推广时，一般流程是：
 - 编辑推荐活动宣传文章 
 - 使用 Web 端、APP 端广告位推广
 - 使用 App 端推送
 - 运营主动邀请活跃用户参与

在邀请活跃用户时，所选择的用户为：
 1. 运营可以方便联系到的
 2. 在社区比较活跃的
 3. 有一定粉丝量的
 
在作出是否邀请某用户时，完全靠运营的主观判断，在这一过程中容易出现：
1. 运营邀请到的用户影响力不足以带来更多参与者
2. 运营在邀请用户参与活动时优先考虑是否方便联系而不是优先考虑被邀请用户的影响力

### 解决办法

1. 根据活动需求筛选一批符合条件的用户
2. 利用 PageRank 算法计算筛选出来的用户各自的影响力
3. 按照影响力高低确定运营主动邀请参与活动的用户名单

In [1]:
import pandas as pd
import networkx as nx
import pymysql
import numpy as np

In [2]:
conn = pymysql.connect(host="*******",user='*****',password="*********",database="*********")

In [3]:
sql = '''
      select followerId,followeeId from **********;
      '''

In [4]:
df = pd.read_sql_query(sql, conn)

从数据库中查询掘金用户的关注关系

In [5]:
df.head()

,followerId,followeeId
0,5aae85666fb9a028e33b3b35,5c936c626fb9a070cb24aee5
1,5aae85666fb9a028e33b3b35,551d677ee4b0cd5b623f49cb
2,5aae85666fb9a028e33b3b35,5c10ad386fb9a049f9126e49
3,5aae85666fb9a028e33b3b35,5cb1529d6fb9a0687d1170ea
4,5aae85666fb9a028e33b3b35,58131b84da2f60005db28799


In [6]:
df.tail()

,followerId,followeeId
132504,5ad56b20f265da23a2299897,5a91037cf265da4e8b2ff784
132505,5ad56b20f265da23a2299897,58b61f4161ff4b006ccba5fd
132506,5ad56b20f265da23a2299897,5ad1d166f265da237b22846c
132507,5ad56b20f265da23a2299897,58afa29aac502e0069c5d756
132508,5ae038c0f265da0b8e7f1237,5ad471ca6fb9a028d82c3dbf


In [7]:
df[df.duplicated()].count()

followerId    0
followeeId    0
dtype: int64

In [8]:
G = nx.DiGraph()
df.apply(lambda row: G.add_edge(row['followerId'], row['followeeId']), axis=1);

In [9]:
pagerank_list = nx.pagerank(G,alpha = 0.01)

In [10]:
df = pd.DataFrame(columns=('ID', 'PageRank'))

In [11]:
for v in pagerank_list:
    df = df.append([{'ID':v,
                    'PageRank':pagerank_list[v]}], ignore_index=True)


In [12]:
df.head()

,ID,PageRank
0,5aae85666fb9a028e33b3b35,0.000045
1,5c936c626fb9a070cb24aee5,0.000045
2,551d677ee4b0cd5b623f49cb,0.000087
3,5c10ad386fb9a049f9126e49,0.000045
4,5cb1529d6fb9a0687d1170ea,0.000044


In [13]:
df.sort_values(by="PageRank" , ascending=False).head(20)

,ID,PageRank
850,nofollowee,0.000308
2,551d677ee4b0cd5b623f49cb,0.000087
167,56fa9a7c2db6ce00478e6bda,0.000074
701,551d6923e4b0cd5b623f54da,0.000069
798,5656fe2260b2febec4b52f46,0.000064
452,555426a7e4b076f1c3243993,0.000063
823,55fbd5e360b249ad605fb5ea,0.000059
807,5699faaf60b219d101745b7b,0.000058
433,55f4419360b28e983c150d0e,0.000058
450,59659aff5188250cf956e6dd,0.000057


至此，选出影响力最大的前20位用户

### 结果

利用 Pagerank 算法可以使得运营在确定参与活动的用户时有数据作为支持，也可以更好的量化后续运营活动的效果。
先挑选一批符合邀请最低要求的用户再计算影响力可以极大的缩短计算时间。
但是相比全量计算所有用户的影响力不足在于，被筛选出来的用户的网络关系有误。
